In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv('dataset/tmdb_5000_movies.csv')
credits = pd.read_csv('dataset/tmdb_5000_credits.csv')

In [3]:
movies = movies.merge(credits,on='title')

In [4]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [5]:
movies.isnull().sum()
movies.dropna(inplace=True)

In [6]:
import ast
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [7]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [8]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter!=3:
            counter +=1 
            L.append(i['name'])
        else:
            break
    return L

In [9]:
movies['cast'] = movies['cast'].apply(convert3)

In [10]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [11]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [12]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [13]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(' ','') for i in x]) 
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(' ','') for i in x]) 
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(' ','') for i in x]) 
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(' ','') for i in x]) 

In [14]:
movies['tags'] = movies['overview']+movies['keywords']+movies['cast']+movies['crew']

In [15]:
new_df = movies[['movie_id','title','tags']]

In [16]:
new_df['tags'] = new_df['tags'].apply(lambda x:' '.join(x))

C:\Users\hp\AppData\Local\Temp\ipykernel_24972\2266712157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:' '.join(x))


In [17]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\hp\AppData\Local\Temp\ipykernel_24972\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [18]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

C:\Users\hp\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [19]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [20]:
cv.fit_transform(new_df['tags']).toarray().shape

(4806, 5000)

In [21]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [22]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [23]:
#cv.get_feature_names()

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
similarty = cosine_similarity(vectors)

In [25]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarty[movie_index]
    movie_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[0:8]
    
    for i in movie_list:
        print(new_df.iloc[i[0]].title)
    

In [26]:
recommend('Avatar')

Avatar
Apollo 18
Aliens vs Predator: Requiem
Lifeforce
Battle: Los Angeles
Titan A.E.
Aliens
Independence Day


In [27]:
import pickle
pickle.dump(new_df, open('model/movies_list.pkl','wb'))

In [28]:
pickle.dump(similarty,open('model/similarity.pkl','wb'))